In [1]:
import polars as pl
from pathlib import Path

# --- CONFIGURAÇÃO ---
# Garante que o Polars mostrará todas as linhas ao imprimir os resultados
pl.Config.set_tbl_rows(-1)

# Caminho para o arquivo de dados. Assumindo que o notebook está na mesma pasta.
DATA_FILE = Path("sih_rs_tratado.parquet")

print(f"Iniciando a análise de discrepância de custos no arquivo: {DATA_FILE}")
print("="*80)

try:
    # 1. Carregar o DataFrame de forma "lazy" para otimização
    lazy_df = pl.scan_parquet(DATA_FILE)

    # 2. Calcular a diferença
    # Garante que valores nulos em VAL_UTI sejam tratados como 0
    df_com_diferenca = lazy_df.with_columns(
        (
            pl.col("VAL_TOT") - (
                pl.col("VAL_SH") + 
                pl.col("VAL_SP") + 
                pl.col("VAL_UTI").fill_null(0)
            )
        ).alias("diferenca")
    )

    # 3. Filtrar apenas as discrepâncias significativas
    # Ignoramos diferenças entre -1 e 1, que são provavelmente erros de arredondamento.
    df_discrepancias = df_com_diferenca.filter(
        pl.col("diferenca").abs() > 1
    )

    # Coleta os resultados para análise
    resultado_discrepancias = df_discrepancias.collect()
    
    # --- APRESENTAÇÃO DOS RESULTADOS ---

    total_registros = lazy_df.select(pl.len()).collect().item()
    total_discrepancias = len(resultado_discrepancias)

    if total_discrepancias == 0:
        print("✅ SUCESSO! Nenhuma discrepância significativa encontrada.")
    else:
        percentual = (total_discrepancias / total_registros) * 100
        print(f"🚨 ALERTA! Encontrados {total_discrepancias:,} registros com discrepância significativa de custos.")
        print(f"   Isso representa {percentual:.2f}% do total de {total_registros:,} registros.")
        print("-" * 50)

        # 4. Agrupar por valor da diferença para ver os mais comuns
        contagem_por_diferenca = (
            resultado_discrepancias
            .group_by("diferenca")
            .agg(pl.len().alias("numero_de_ocorrencias"))
            .sort("numero_de_ocorrencias", descending=True)
        )

        print("\nValores de diferença mais comuns:")
        print(contagem_por_diferenca.head(25)) # Mostra os 25 valores de diferença mais frequentes

        # 5. Mostrar um resumo estatístico completo da coluna 'diferenca'
        resumo_estatistico = resultado_discrepancias.select(
            pl.col("diferenca")
        ).describe()

        print("\nResumo estatístico da coluna 'diferenca' (para os registros com discrepância):")
        print(resumo_estatistico)
        
        # 6. Mostrar uma amostra dos dados com as maiores discrepâncias
        amostra_maiores_discrepancias = (
            resultado_discrepancias
            .sort("diferenca", descending=True)
            .head(10)
            .select(["N_AIH", "VAL_TOT", "VAL_SH", "VAL_SP", "VAL_UTI", "diferenca"])
        )
        
        print("\nAmostra dos 10 registros com as MAIORES diferenças positivas:")
        print(amostra_maiores_discrepancias)


except FileNotFoundError:
    print(f"\nERRO: Arquivo '{DATA_FILE}' não encontrado. Verifique o caminho.")
except Exception as e:
    print(f"\nOcorreu um erro inesperado: {e}")



Iniciando a análise de discrepância de custos no arquivo: sih_rs_tratado.parquet
🚨 ALERTA! Encontrados 2,014,472 registros com discrepância significativa de custos.
   Isso representa 8.47% do total de 23,792,498 registros.
--------------------------------------------------

Valores de diferença mais comuns:
shape: (25, 2)
┌───────────┬───────────────────────┐
│ diferenca ┆ numero_de_ocorrencias │
│ ---       ┆ ---                   │
│ f64       ┆ u32                   │
╞═══════════╪═══════════════════════╡
│ -957.44   ┆ 62824                 │
│ -478.72   ┆ 51378                 │
│ -1436.16  ┆ 50484                 │
│ -508.63   ┆ 46444                 │
│ -1017.26  ┆ 41262                 │
│ -1914.88  ┆ 39250                 │
│ -478.72   ┆ 33974                 │
│ -1436.16  ┆ 30438                 │
│ -3351.04  ┆ 24836                 │
│ -957.44   ┆ 23914                 │
│ -2393.6   ┆ 23096                 │
│ -1525.89  ┆ 22692                 │
│ -2872.32  ┆ 20102          

In [1]:
import pandas as pd

df = pd.read_parquet("sih_rs_tratado.parquet")
print(df["N_AIH"])

0           2108100145052
1           2108100145063
2           2108100145074
3           2108100145085
4           2108100145130
                ...      
37342653    4323108845760
37342654    4323108845770
37342655    4323108845792
37342656    4323108845803
37342657    4323108845836
Name: N_AIH, Length: 37342658, dtype: object


In [3]:
import pandas as pd

# Carregar parquet tratado
df = pd.read_parquet("sih_rs_tratado.parquet")

# Filtrar apenas os procedimentos desejados
codigos = ["505020092", "505020106"]
df_transplantes = df[df["PROC_REA"].isin(codigos)]

# Manter apenas as colunas que o dash usa
colunas_dash = ["N_AIH", "PROC_REA", "IDADE", "SEXO", "CID_MORTE", "VAL_TOT", "DT_INTER", "DT_SAIDA", "MUNIC_MOV"]
df_transplantes = df_transplantes[colunas_dash]

# Salvar em parquet
df_transplantes.to_csv("transplantes_filtrados.csv", index=False)

print(" Arquivo 'transplantes_filtrados.csv' gerado com sucesso!")


 Arquivo 'transplantes_filtrados.csv' gerado com sucesso!


In [6]:
import polars as pl
from pathlib import Path

# --- CONFIGURAÇÃO ---
DATA_FILE = Path("sih_rs_tratado.parquet")

print(f"Gerando Relatório de Qualidade de Custos para o arquivo: {DATA_FILE}")
print("="*80)

try:
    # Usamos scan_parquet para ser eficiente com a memória, pois só faremos contagens
    lazy_df = pl.scan_parquet(DATA_FILE)
    total_registros = lazy_df.select(pl.len()).collect().item()
    print(f"Arquivo analisado. Total de registros: {total_registros:,}")
    print("="*80)

    # ==================================================
    # MÉTRICA 1: VALORES DISCREPANTES
    # ==================================================
    
    # Define a expressão para a diferença
    expr_diferenca = (
        pl.col("VAL_TOT") - (
            pl.col("VAL_SH") + 
            pl.col("VAL_SP") + 
            pl.col("VAL_UTI").fill_null(0)
        )
    )

    # Conta quantos registros têm uma diferença significativa (maior que 1)
    contagem_discrepancia = lazy_df.filter(
        expr_diferenca.abs() > 1
    ).select(
        pl.len()
    ).collect().item()

    # ==================================================
    # MÉTRICA 2: VALORES NEGATIVOS
    # ==================================================

    # Conta quantos registros têm pelo menos um valor de custo negativo
    contagem_negativos = lazy_df.filter(
        (pl.col("VAL_TOT") < 0) |
        (pl.col("VAL_SH") < 0) |
        (pl.col("VAL_SP") < 0) |
        (pl.col("VAL_UTI") < 0)
    ).select(
        pl.len()
    ).collect().item()

    # ==================================================
    # APRESENTAÇÃO DO RELATÓRIO FINAL
    # ==================================================
    
    percentual_disc = (contagem_discrepancia / total_registros) * 100
    percentual_neg = (contagem_negativos / total_registros) * 100

    print("\n--- RELATÓRIO DE QUALIDADE DE DADOS DE CUSTOS ---\n")
    print(f"1. Registros com Soma de Custos Discrepante:")
    print(f"   - Total: {contagem_discrepancia:,} registros")
    print(f"   - Percentual do Dataset: {percentual_disc:.2f}%\n")
    
    print(f"2. Registros com Pelo Menos Um Valor de Custo Negativo:")
    print(f"   - Total: {contagem_negativos:,} registros")
    print(f"   - Percentual do Dataset: {percentual_neg:.2f}%\n")

    print("--------------------------------------------------")

except FileNotFoundError:
    print(f"\nERRO: Arquivo '{DATA_FILE}' não encontrado. Verifique o caminho.")
except Exception as e:
    print(f"\nOcorreu um erro inesperado: {e}")



Gerando Relatório de Qualidade de Custos para o arquivo: sih_rs_tratado.parquet
Arquivo analisado. Total de registros: 23,792,498

--- RELATÓRIO DE QUALIDADE DE DADOS DE CUSTOS ---

1. Registros com Soma de Custos Discrepante:
   - Total: 0 registros
   - Percentual do Dataset: 0.00%

2. Registros com Pelo Menos Um Valor de Custo Negativo:
   - Total: 0 registros
   - Percentual do Dataset: 0.00%

--------------------------------------------------


In [8]:
import polars as pl
from pathlib import Path

# --- CONFIGURAÇÃO ---
# Caminho para o arquivo que sai do pré-processamento
PREPROCESSED_FILE = Path("sih_rs_tratado.parquet")

print(f"Iniciando verificação de consistência interna do arquivo: '{PREPROCESSED_FILE}'")
print("="*80)

try:
    # Usamos scan_parquet para ser eficiente com a memória
    lazy_df = pl.scan_parquet(PREPROCESSED_FILE)
    total_registros = lazy_df.select(pl.len()).collect().item()
    print(f"Arquivo encontrado. Total de registros a serem verificados: {total_registros:,}")

    # --- Verificação de Consistência ---
    
    # Filtra o DataFrame para encontrar linhas onde DIAS_PERM não bate com o cálculo das datas
    # Usamos .dt.total_days() para extrair o número de dias da diferença
    df_inconsistentes = lazy_df.filter(
        pl.col("DIAS_PERM") != (pl.col("DT_SAIDA") - pl.col("DT_INTER")).dt.total_days()
    ).collect() # .collect() executa a consulta e materializa o resultado

    total_inconsistentes = len(df_inconsistentes)

    print("-" * 80)
    if total_inconsistentes == 0:
        print("\n✅ SUCESSO! O arquivo 'sih_rs_tratado.parquet' está 100% consistente.")
        print("   A coluna 'DIAS_PERM' corresponde perfeitamente ao cálculo (DT_SAIDA - DT_INTER).")
        print("   Isso sugere que a inconsistência está sendo introduzida na etapa de AGREGAÇÃO ('aggregate.py').")
    else:
        print(f"\n🚨 ALERTA! Encontrados {total_inconsistentes:,} registros inconsistentes dentro do próprio arquivo 'sih_rs_tratado.parquet'.")
        print("   Isso indica um problema na lógica de recálculo dentro do 'preprocess.py'.")
        
        # Adiciona a coluna de diferença para facilitar a análise
        df_inconsistentes = df_inconsistentes.with_columns(
            (pl.col("DIAS_PERM") - (pl.col("DT_SAIDA") - pl.col("DT_INTER")).dt.total_days()).alias("diferenca")
        )
        
        print("\n--- Amostra dos dados inconsistentes encontrados ---")
        print(df_inconsistentes.select([
            "N_AIH", "DT_INTER", "DT_SAIDA", "DIAS_PERM", "diferenca"
        ]).head(20))

except FileNotFoundError:
    print(f"\nERRO: Arquivo '{PREPROCESSED_FILE}' não encontrado. Verifique o caminho.")
except Exception as e:
    print(f"\nOcorreu um erro inesperado: {e}")



Iniciando verificação de consistência interna do arquivo: 'sih_rs_tratado.parquet'
Arquivo encontrado. Total de registros a serem verificados: 23,792,498
--------------------------------------------------------------------------------

✅ SUCESSO! O arquivo 'sih_rs_tratado.parquet' está 100% consistente.
   A coluna 'DIAS_PERM' corresponde perfeitamente ao cálculo (DT_SAIDA - DT_INTER).
   Isso sugere que a inconsistência está sendo introduzida na etapa de AGREGAÇÃO ('aggregate.py').


In [2]:
import polars as pl

# Substitua 'caminho/do/seu/arquivo.parquet' pelo caminho correto
df = pl.read_parquet('sih_rs_tratado.parquet', columns=["N_AIH", "RACA_COR", "ETNIA"])

# Filtra apenas por RACA_COR == '5'
df_etnia = df.filter(pl.col("RACA_COR") == "5")

# Imprime os 10 primeiros registros para inspeção
print(df_etnia.head(10))

# Imprime os valores únicos da coluna ETNIA para RACA_COR == '5'
print(df_etnia.select('ETNIA').unique())

shape: (0, 3)
┌───────┬──────────┬───────┐
│ N_AIH ┆ RACA_COR ┆ ETNIA │
│ ---   ┆ ---      ┆ ---   │
│ str   ┆ str      ┆ i8    │
╞═══════╪══════════╪═══════╡
└───────┴──────────┴───────┘
shape: (0, 1)
┌───────┐
│ ETNIA │
│ ---   │
│ i8    │
╞═══════╡
└───────┘


In [1]:
import polars as pl

# Lê os dois Parquets
original = pl.scan_parquet("sih_rs_tratado.parquet").collect()
tratado = pl.scan_parquet("sih_rs_variavel_tipo.parquet").collect()

# Define a chave composta
chave = ["N_AIH", "PROC_REA", "CNES", "DT_INTER", "DT_SAIDA"]

# Garante que as colunas de chave existam (descarta as ausentes)
chave_validas = [c for c in chave if c in original.columns and c in tratado.columns]

# Cria join baseado nas chaves válidas
df_join = original.join(tratado, on=chave_validas, how="inner", suffix="_trat")

# Compara coluna a coluna, exceto as chaves
cols_comuns = [
    c for c in original.columns
    if c in tratado.columns and c not in chave_validas
]

comparacao = []
for col in cols_comuns:
    iguais = (df_join[col] == df_join[f"{col}_trat"]).sum()
    total = len(df_join)
    diferentes = total - iguais
    comparacao.append({
        "coluna": col,
        "iguais": iguais,
        "diferentes": diferentes,
        "porcentagem_iguais": round(iguais / total * 100, 2)
    })

# Resultado final
df_resultado = pl.DataFrame(comparacao).sort("porcentagem_iguais", descending=True)
df_resultado.write_csv("comparacao_preprocessamento.csv")

print(df_resultado)
print(f"\nTotal de registros comparados: {len(df_join):,}")


ZeroDivisionError: division by zero